In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql import functions as F
import pandas as pd
from sqlalchemy import create_engine
sc = SparkContext()
spark = SparkSession(sc)

In [2]:
engine = create_engine('mysql+mysqlconnector://user:u53r_mysql@db/article_clustering')
article_frame = spark.createDataFrame(pd.read_sql('article', engine, columns=['vector', 'id']))

In [3]:
new_schema = ArrayType(DoubleType(), containsNull=False)
udf_foo = F.udf(lambda x: x, new_schema)
article_frame = article_frame.withColumn("vector", udf_foo("vector"))
bkm = BisectingKMeans(k=100, featuresCol='vector')
model = bkm.fit(article_frame)
model.setPredictionCol("cluster_id")
article_frame = model.transform(article_frame)

In [4]:
article_frame = article_frame.toPandas()
cluster_frame = pd.Series(article_frame['cluster_id'].unique())

In [5]:
article_frame['article_id'] = article_frame['id']
article_frame = article_frame[['article_id', 'cluster_id']]
article_frame

,article_id,cluster_id
0,2342,82
1,2343,95
2,2344,97
3,2345,96
4,2346,82
...,...,...
3100,5442,90
3101,5443,75
3102,5444,80
3103,5445,87


In [6]:
cluster_frame = pd.DataFrame(article_frame['cluster_id'].unique(), columns=['id'])

In [7]:
cluster_frame

,id
0,82
1,95
2,97
3,96
4,89
...,...
95,86
96,30
97,10
98,3


In [8]:
engine.execute('DELETE FROM cluster')
cluster_frame.to_sql('cluster', engine, if_exists='append', index=False)

In [9]:
article_frame.to_sql('article_cluster', engine, if_exists='append', index=False)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 48754)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/lib/python3.8/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/usr/lib/python3.8/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/python3.8/site-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lib/python3.8/site-p